In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from dask.distributed import Client
from distributed.deploy.local import LocalCluster
from pycalphad import Database, equilibrium, variables as v
from pduq.invariant_calc import invariant_samples
from pduq.dbf_calc import eq_calc_samples
from pduq.uq_plot import plot_contour
sns.set(color_codes=True)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
c = LocalCluster(n_workers=8, threads_per_worker=1)
client = Client(c)
print(client)

In [ ]:
dbf = Database('ag-co_dft.tdb')
params = np.load('trace.npy')[:, -1, :]
conds = {v.P: 101325, v.T: (300, 4000, 10), v.X('CO'): (0, 1, 0.01)}

# perform the equilibrium calculation
eq = eq_calc_samples(dbf, conds, params, client=client)

相边界不确定性

In [ ]:
phaseL = list(np.unique(eq.get('Phase').values))
if '' in phaseL: phaseL.remove('')
nph = len(phaseL)
colorL = sns.color_palette("cubehelix", nph+2)
cdict = {}
for ii in range(nph):
    cdict[phaseL[ii]] = colorL[ii]

# we can also specify a dictionary to have pretty
# labels (even LaTeX) in our plots
phase_label_dict = {
    'LIQUID':'liquid', 'FCC_A1':'FCC', 'HCP_A3':'HCP'}

In [ ]:
def plot_binary(eq, comp, alpha=None, cdict=None):

    """
    Plot a binary phase diagram. This purposefully has a
    minimal number of options so that the returned figure
    can be customized easily.

    Parameters
    ----------
    eq : xarray object
        Structured equilibirum calculation
    comp : str
        Label for species to plot on the x-axis,
        e.g. MG for magnesium
    alpha : float, optional
        Number between 0 and 1 for the line transparency
    cdict : dict, optional
        Dictionary with phase names and corresponding
        colors

    Returns
    -------
    compL : numpy array
        1D array with typ values for all parameter sets where
        only the phases in phaseregL are in equilibrium

    Examples
    --------
    >>> import pickle
    >>> import pduq.uq_plot as uq
    >>> with open('single.pkl', 'rb') as buff:
    >>>     eq = pickle.load(buff)
    >>> comp = 'MG'
    >>> # plot a binary phase diagram for a set of
    >>> # equilibrium calculations, and comp as the
    >>> # molar fraction on the x-axis
    >>> uq.plot_binary(eq, comp)
    """

    Tvec = eq.get('T').values  # all temperature values
    Xvec = eq.get('X_' + comp).values  # all molar composition values

    # phaseL: list of unique phases
    phaseL = list(np.unique(eq.get('Phase').values))
    if '' in phaseL:
        phaseL.remove('')
    nph = len(phaseL)  # number of unique phases

    Xph = np.zeros((Tvec.size, Xvec.size, nph))

    for ii in range(nph):
        tmp = eq.X.where(eq.Phase == phaseL[ii])
        tmp = tmp.sel(component=comp)

        nans = np.squeeze(tmp.values)
        nans = np.isnan(nans[..., 0])*np.isnan(nans[..., 1])
        Xph_ = np.squeeze(tmp.sum(dim='vertex').values)
        Xph_[nans] = np.nan
        Xph[..., ii] = Xph_

    XphD = {}

    for ii in range(Tvec.size):
        T = str(np.int32(Tvec[ii]))
        xl = 'X_' + T
        pl = 'Ph_' + T
        XphD[xl], XphD[pl] = [], []
        for jj in range(nph):
            # find values not equal to the composition
            neq2comp = np.isclose(Xph[ii, :, jj], Xvec, atol=1e-4)
            neq2comp = np.invert(neq2comp)
            vals = Xph[ii, neq2comp, jj]

            # remove nans
            vals = vals[np.invert(np.isnan(vals))]

            # get unique values
            vals_ = np.round(vals, 5)
            loc, indx = np.unique(vals_, return_index=True)
            vals = vals[indx]

            XphD[xl] += list(vals)
            XphD[pl] += len(vals)*[phaseL[jj]]

        arg = np.argsort(np.array(XphD[xl]))
        XphD[xl] = np.array(XphD[xl])[arg]
        XphD[pl] = np.array(XphD[pl])[arg]

    colorL = sns.color_palette("cubehelix", nph)
    if alpha is None:
        alpha = 0.9

    for ii in range(Tvec.size):
        T = str(np.int32(Tvec[ii]))
        xl = 'X_' + T
        pl = 'Ph_' + T
        vals = XphD[xl]
        phs = XphD[pl]
        for jj in range(nph):
            if phaseL[jj] not in list(phs):
                continue
            vals_ = vals[phs == phaseL[jj]]

            if cdict is not None:
                color = cdict[phaseL[jj]]
            else:
                color = colorL[jj]

            plt.plot(vals_, [Tvec[ii]]*len(vals_),
                     marker='o', markersize=2, alpha=alpha,
                     color=color, linestyle='', mew=0.0)

In [ ]:
def plot_superimposed(
        eq, comp, nsp=None, alpha=None,
        phase_label_dict=None,
        xlims=None, cdict=None, figsize=None):
    phaseL = list(np.unique(eq.get('Phase').values))
    if '' in phaseL:
        phaseL.remove('')
    nph = len(phaseL)

    if nsp is None:
        nsp = len(eq.sample)

    fig, ax = plt.subplots(figsize=figsize)

    # plot the phase boundaries for each parameter set
    for ii in range(nsp):
        plot_binary(
            eq.sel(sample=ii), comp=comp, alpha=alpha, cdict=cdict)
        print('diagram', ii, 'plotted')

    # plot the legend
    nph = len(phaseL)
    colorL = sns.color_palette("cubehelix", nph)
    Tvec = eq.get('T').values
    for ii in range(nph):
        phase = phaseL[ii]
        if cdict is not None:
            color = cdict[phaseL[ii]]
        else:
            color = colorL[ii]
        if phase_label_dict is not None:
            label = phase_label_dict[phase]
        else:
            label = phase
        plt.plot(1.5, Tvec[0], color=color, linestyle='',
                 marker='.', label=label)

    if xlims is None:
        Xvec = eq.get('X_' + comp).values
        Xrng = Xvec.max() - Xvec.min()
        xlims = [Xvec.min() - 0.005*Xrng, Xvec.max() + 0.005*Xrng]

    plt.xlim(xlims)
    plt.ylim([300, 4000])
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel(r'$\mathrm{x_{%s}}$' % comp, fontsize='large')
    plt.ylabel('T (K)', fontsize='large')
    plt.legend()
    plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
plot_superimposed(eq, 'CO', alpha=0.2,
                  xlims=[0.9995, 1], cdict=cdict,
                  phase_label_dict=phase_label_dict,
                  figsize=(5, 3))
plt.legend().remove()
plt.savefig('Phase_Diagram_Uncertainty1.png',dpi=600)

In [ ]:
import matplotlib.pyplot as plt
plot_superimposed(eq, 'CO', alpha=0.2,
                  xlims=[0.999995, 1], cdict=cdict,
                  phase_label_dict=phase_label_dict,
                  figsize=(5, 3))
plt.legend().remove()
plt.savefig('Phase_Diagram_Uncertainty2.png',dpi=600)

In [ ]:
from pduq.uq_plot import plot_phasereg_prob
plot_phasereg_prob(
    eq, ['LIQUID'], coordplt=['X_CO', 'T'], figsize=(5, 3))
plt.savefig('Phase_Diagram_Uncertainty00001.png',dpi=600)

In [ ]:
plot_phasereg_prob(
    eq, ['FCC_A1'], coordplt=['X_CO', 'T'], figsize=(5, 3))
plt.savefig('Phase_Diagram_Uncertainty00002.png',dpi=600)

相分数

In [ ]:
phaseL = list(np.unique(eq.get('Phase').values))
if '' in phaseL: phaseL.remove('')
nph = len(phaseL)
colorL = sns.color_palette("cubehelix", nph+2)
cdict = {}
for ii in range(nph):
    cdict[phaseL[ii]] = colorL[ii]

# we can also specify a dictionary to have pretty
# labels (even LaTeX) in our plots
phase_label_dict = {
    'LIQUID':'liquid', 'FCC_A1':'FCC', 'HCP_A3':'HCP'}

In [ ]:
from pduq.uq_plot import plot_phasefracline
import matplotlib.pyplot as plt
def plot_phasefracline(eq, coordD, xlabel=None,
                       phase_label_dict=None, cdict=None, figsize=None):

    """
    Plot the phase fraction with uncertainty versus composition,
    temperature or pressure.

    Parameters
    ----------
    eq : xarray object
        Structured equilibirum calculation containing a 'sample'
        dimension correspoinding to different parameter sets
    coordD : dict
        Dictionary defining constraints on the coordinates in
        eq for plotting the phase fraction with varying X, T or P
        For example, we might pick a fixed X and let T vary
        as follows: coordD = {'X_MG':0.1}
    xlabel : str, optional
        Label for the x-axis
    phase_label_dict : dict, optional
        Dictionary with keys given by phase names and corresponding
        strings to use in plotting (e.g. to enable LaTeX labels)
    xlims : list or tuple of float, optional
        List or tuple with two floats corresponding to the
        minimum and maximum molar composition of comp
    cdict : dict, optional
        Dictionary with phase names and corresponding
        colors
    figsize : tuple or list of int or float, optional
        Plot dimensions in inches

    Returns
    -------

    Examples
    --------
    >>> import pickle
    >>> import pduq.uq_plot as uq
    >>> with open('full.pkl', 'rb') as buff:
    >>>     eq = pickle.load(buff)
    >>> # if for 'full.pkl' X_MG and T have 100 intervals each
    >>> # we can fix T and plot the phase fraction versus
    >>> # X_MG with uncertainty
    >>> coordD = {'T':1000}
    >>> # plot phase fraction versus X_MG
    >>> uq.plot_phasefracline(eq, coordD, xlabel='X_MG')
    """
    phaseL = list(np.unique(eq.get('Phase').values))
    if '' in phaseL:
        phaseL.remove('')
    nph = len(phaseL)

    eq = eq.sel(coordD)

    # the X, T or P dimension with the most values
    # is then plotted on the x-axis
    rmlist = ['N', 'internal_dof', 'sample', 'vertex']
    max_sz = 0
    for dim in list(eq.dims):
        if dim not in rmlist:
            dim_sz = eq.dims[dim]
            if dim_sz > max_sz:
                max_sz = dim_sz
                dim_max = dim
    xvec = eq.get(dim_max).values

    CI = 95
    colorL = sns.color_palette("cubehelix", nph)

    plt.figure(figsize=figsize)

    for ii in range(nph):
        phase = phaseL[ii]
        val = eq.NP.where(eq.Phase == phase)
        val = val.sum(dim='vertex').values.squeeze()

        low, mid, high = np.percentile(
            val, [0.5*(100-CI), 50, 100-0.5*(100-CI)], axis=0)

        if phase_label_dict is not None:
            label = phase_label_dict[phase]
        else:
            label = phase

        if cdict is not None:
            color = cdict[phaseL[ii]]
        else:
            color = colorL[ii]

        plt.plot(xvec, mid, linestyle='-', color=color, label=label)
        plt.fill_between(
            np.atleast_1d(xvec), low, high, alpha=0.3, facecolor=color)
    
    #plt.grid(True)

    plt.xlim([1200, 1250])
    plt.ylim([-0.01, 1.01])
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel(xlabel, fontsize='large')
    plt.ylabel('phase fraction', fontsize='large')
    plt.legend()
    plt.tight_layout()

In [ ]:
#from pduq.uq_plot import plot_phasefracline
# define a dictionary fixing the composition. We need to pick
# a value for X_MG that is represented in the eq object
coordD = {'X_CO':0.1}

# plot the phase fraction versus composition
# cdict and phase_label_dict are the same as in the previous example
plot_phasefracline(eq, coordD, xlabel='T (K)', cdict=cdict,
                      phase_label_dict=phase_label_dict, figsize=(5, 3))
plt.savefig('phase_fraction.png',dpi=600)

In [ ]:
def plot_phasefracline(eq, coordD, xlabel=None,
                       phase_label_dict=None, cdict=None, figsize=None):

    """
    Plot the phase fraction with uncertainty versus composition,
    temperature or pressure.

    Parameters
    ----------
    eq : xarray object
        Structured equilibirum calculation containing a 'sample'
        dimension correspoinding to different parameter sets
    coordD : dict
        Dictionary defining constraints on the coordinates in
        eq for plotting the phase fraction with varying X, T or P
        For example, we might pick a fixed X and let T vary
        as follows: coordD = {'X_MG':0.1}
    xlabel : str, optional
        Label for the x-axis
    phase_label_dict : dict, optional
        Dictionary with keys given by phase names and corresponding
        strings to use in plotting (e.g. to enable LaTeX labels)
    xlims : list or tuple of float, optional
        List or tuple with two floats corresponding to the
        minimum and maximum molar composition of comp
    cdict : dict, optional
        Dictionary with phase names and corresponding
        colors
    figsize : tuple or list of int or float, optional
        Plot dimensions in inches

    Returns
    -------

    Examples
    --------
    >>> import pickle
    >>> import pduq.uq_plot as uq
    >>> with open('full.pkl', 'rb') as buff:
    >>>     eq = pickle.load(buff)
    >>> # if for 'full.pkl' X_MG and T have 100 intervals each
    >>> # we can fix T and plot the phase fraction versus
    >>> # X_MG with uncertainty
    >>> coordD = {'T':1000}
    >>> # plot phase fraction versus X_MG
    >>> uq.plot_phasefracline(eq, coordD, xlabel='X_MG')
    """
    phaseL = list(np.unique(eq.get('Phase').values))
    if '' in phaseL:
        phaseL.remove('')
    nph = len(phaseL)

    eq = eq.sel(coordD)

    # the X, T or P dimension with the most values
    # is then plotted on the x-axis
    rmlist = ['N', 'internal_dof', 'sample', 'vertex']
    max_sz = 0
    for dim in list(eq.dims):
        if dim not in rmlist:
            dim_sz = eq.dims[dim]
            if dim_sz > max_sz:
                max_sz = dim_sz
                dim_max = dim
    xvec = eq.get(dim_max).values

    CI = 95
    colorL = sns.color_palette("cubehelix", nph)

    plt.figure(figsize=figsize)

    for ii in range(nph):
        phase = phaseL[ii]
        val = eq.NP.where(eq.Phase == phase)
        val = val.sum(dim='vertex').values.squeeze()

        low, mid, high = np.percentile(
            val, [0.5*(100-CI), 50, 100-0.5*(100-CI)], axis=0)

        if phase_label_dict is not None:
            label = phase_label_dict[phase]
        else:
            label = phase

        if cdict is not None:
            color = cdict[phaseL[ii]]
        else:
            color = colorL[ii]

        plt.plot(xvec, mid, linestyle='-', color=color, label=label)
        plt.fill_between(
            np.atleast_1d(xvec), low, high, alpha=0.3, facecolor=color)
    
    #plt.grid(True)

    plt.xlim([0, 1])
    plt.ylim([-0.01, 1.01])
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel(xlabel, fontsize='large')
    plt.ylabel('phase fraction', fontsize='large')
    plt.legend()
    plt.tight_layout()

In [ ]:
coordD = {'T':1500}

# plot the phase fraction versus composition
# cdict and phase_label_dict are the same as in the previous example
plot_phasefracline(eq, coordD, xlabel=r"$x_{Co}$", cdict=cdict,
                      phase_label_dict=phase_label_dict, figsize=(5, 3))
plt.savefig('phase_fraction_1500.png',dpi=600)

热力学性质不确定性

In [ ]:
from espei.core_utils import get_data#(0.8.3版本需要使用该函数)
#from espei.core_utils import filter_configurations #(大于0.8.3版本需要使用该函数)
from pycalphad import calculate
from espei.utils import database_symbols_to_fit
from collections import OrderedDict
import xarray as xr

In [ ]:
from pduq.uq_plot import plot_property
comps = ['AG', 'CO', 'VA']  # species to consider
T = 1739  # temperature in Kelvin
prop = 'GM'  # property of interest (molar Gibbs energy)
ylabel = 'Molar Gibbs energy\n(kJ/mol.)'  # y-axis label
yscale = 1e-3  # we want to scale the Gibbs energy to give kJ/mol.

# we can now plot the property. Note that phaseL, phase_label_dict,
# and cdict are defined in the phase diagram prediction example
plot_property(dbf, comps, phaseL, params, T, prop,
                 phase_label_dict=phase_label_dict,
                 ylabel=ylabel, yscale=yscale, xlabel=r"$x_{Co}$", cdict=cdict,
                 xlim=[-0.005, 1.005], figsize=(5, 3))
plt.savefig('1739Molar-Gibbs-energy1.png',dpi=600)

In [ ]:
from pduq.uq_plot import plot_property
comps = ['AG', 'CO', 'VA']  # species to consider
T = 1739  # temperature in Kelvin
prop = 'HM_MIX'  # property of interest (molar Gibbs energy)
ylabel = 'Molar enthalpy of mixing\n(kJ/mol.)'  # y-axis label
yscale = 1e-3  # we want to scale the Gibbs energy to give kJ/mol.

# we can now plot the property. Note that phaseL, phase_label_dict,
# and cdict are defined in the phase diagram prediction example
plot_property(dbf, comps, phaseL, params, T, prop,
                 phase_label_dict=phase_label_dict,
                 ylabel=ylabel, yscale=yscale, xlabel=r"$x_{Co}$", cdict=cdict,
                 xlim=[-0.005, 1.005], figsize=(5, 3))
plt.savefig('1739Molar-enthalpy-of-mixing.png',dpi=600)

相变点概率分布

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from dask.distributed import Client
from distributed.deploy.local import LocalCluster
from pycalphad import Database, equilibrium, variables as v
from pduq.invariant_calc import invariant_samples
from pduq.uq_plot import plot_contour
sns.set(color_codes=True)
import warnings
warnings.filterwarnings("ignore")
c = LocalCluster(n_workers=8, threads_per_worker=1)
client = Client(c)
print(client)
dbf = Database('ag-co_dft.tdb')
params = np.load('trace.npy')[:, -1, :]

In [ ]:
X = 0.004  # X_MG guess for invariant
P = 101324  # pressure
Tl = 1100  # lower temperature bound for invariant
Tu = 1300  # upper temperature bound for invariant
comp = 'CO' # species to reference for composition

In [ ]:
import logging
import pickle
import sympy
import numpy as np
import xarray as xr
from collections import OrderedDict
from espei.utils import database_symbols_to_fit
from itertools import chain
from pycalphad import equilibrium, variables as v
from pycalphad.codegen.callables import build_callables
from pycalphad.core.utils import instantiate_models
from time import time
import logging
import numpy as np
#from .dbf_calc import eq_calc_  # , get_eq_callables_
from espei.utils import database_symbols_to_fit
from pycalphad import variables as v

comps = list(dbf.elements)
phases = list(dbf.phases.keys())
neq = params.shape[0] # calculate invariants for neq parameter sets
bndv = np.zeros((neq, 2))



symbols_to_fit = database_symbols_to_fit(dbf)

# eq_callables = get_eq_callables_(dbf, comps, phases, symbols_to_fit)
eq_callables = None  # eq_callables is disabled for current pycalcphad
kwargs = {'dbf': dbf, 'comps': comps, 'phases': phases,
              'X': X, 'P': P, 'Tl': Tl, 'Tu': Tu, 'comp': comp,
              'params': params, 'symbols_to_fit': symbols_to_fit,
              'eq_callables': eq_callables}

In [ ]:
def eq_calc_(dbf, comps, phases, conds,
             paramA, symbols_to_fit,
             eq_callables=None):

    param_dict = {param_name: param for param_name, param
                  in zip(symbols_to_fit, paramA)}

    parameters = OrderedDict(sorted(param_dict.items(), key=str))

    eq_result = equilibrium(dbf, comps, phases, conds,
                            parameters=parameters, callables=eq_callables)

    return eq_result

In [ ]:
def invariant_(index, dbf, params, comps, phases, X, P, Tl, Tu, comp,
               symbols_to_fit, eq_callables):

    kwargs = {'dbf': dbf, 'comps': comps, 'phases': phases,
              'paramA': params[index, :], 'symbols_to_fit': symbols_to_fit,
              'eq_callables': eq_callables}

    def mini(T):
        """
        perform the equilibrium calculation at a specified temperature
        and return the list of unique phases in equilibrium along with the
        equlibrium data object
        """
        conds = {v.P: P, v.T: T, v.X(comp): X}
        eq = eq_calc_(conds=conds, **kwargs)
        PhT = list(np.unique(eq.Phase))
        if '' in PhT:
            PhT.remove('')
        return eq, PhT

    # perform equilibrium calculations at the lower bound, middle, and
    # upper bound temperatures
    Tm = 0.5*(Tl+Tu)  # middle temperature
    eql, PhTl = mini(Tl)
    eqm, PhTm = mini(Tm)
    equ, PhTu = mini(Tu)

    # now we use the bisection method to find the invariant temperature
    # and composition

    # identify the number of calculations so that the error in the
    # temperature is less than errlim
    errlim = 0.001
    niter = np.log(errlim/(Tu-Tl))/np.log(0.5) - 1
    niter = np.int16(np.ceil(niter))

    for ii in range(niter):
        # if the phases in equilibrium at the middle temperature are
        # different than at the lower temperature, then the invariant
        # will be between the lower and middle temperature. We can then
        # set the upper temperature to the previous middle temperature.
        # Otherwise, the invariant will be between the middle and upper
        # temperature.
        if str(PhTm) != str(PhTl):
            Tu = Tm
            equ = eqm
            PhTu = PhTm
        else:
            Tl = Tm
            eql = eqm
            PhTl = PhTm
        Tm = 0.5*(Tl + Tu)  # get the new middle temperature
        eqm, PhTm = mini(Tm)  # calculate the Tm equilibrium
        # print(Tm, ' ', PhTm)

    def getbnd(eq, PhT):
        """
        get the molar compositions of the phases in PhT
        """
        bnd = []
        for phase in PhT:
            tmp = eq.X.where(eq.Phase == phase)
            tmp = tmp.sel(component=comp).sum(dim='vertex')
            bnd.append(np.squeeze(tmp.values))
        return np.array(bnd)

    # PhTA are the phases at Tl and Tu
    PhTA = np.concatenate([PhTl, PhTu])
    # bndA are the molar compositions of the phases in PhTl and PhTu
    bndA = np.concatenate([getbnd(eql, PhTl), getbnd(equ, PhTu)])
    # phs are the unique phases in the three phase region
    phs, indx = np.unique(PhTA, return_index=True)
    # bnd are the molar compositions corresponding to phs
    bnd = bndA[indx]

    indx = np.argsort(bnd)
    phs = list(phs[indx])
    bnd = bnd[indx]

    logging.info('Invariant computed for set ' + str(index))
    logging.info('T = ' + str(Tm) + ' +/- ' + str(Tm-Tl) + 'K')
    logging.info(str(phs))
    logging.info(str(bnd))

    # print('Invariant computed for set ', index)
    # print('T=', Tm, '+/-', Tm-Tl, 'K')
    #print(phs)
    # print(bnd)

    return Tm, phs, bnd, index

In [ ]:
invL = []
for ii in range(neq):
    invL.append(invariant_(ii, **kwargs))

In [ ]:
print(invL[5][0])
print(invL[5][1])
print(invL[11][2])

In [ ]:
Tv = np.zeros((neq,))
phv = neq*[None]
bndv = np.zeros((neq, 2))
for ii in range(neq):
    Tv[ii] = invL[ii][0]
    phv[ii] = invL[ii][1]
    bndv[ii] = invL[ii][2]

In [ ]:
print(Tv)

In [ ]:
print(bndv)

In [ ]:
Tv=[1231.27027378,1231.52172565,1230.32912998,1231.21777096,1231.24247112
,1231.73880367,1231.51360798,1231.63061657,1231.30668964,1231.51600666
,1230.26857891,1230.21445065,1231.27302914,1230.50668964,1230.3502254
,1231.52835674,1230.66498089,1231.69286251,1231.71946583,1230.15270023
,1231.21953526,1231.78375378,1230.71494617,1230.96604137,1230.94317493
,1231.68947277,1230.80189228,1231.5981554,1230.70252666,1230.66194782
,1231.87888699,1231.1678751,1230.80365658,1231.6623045,1231.82066517
,1230.68368435,1229.9479723,1230.77189922,1231.44945889,1230.92842617
,1230.72426319,1230.67076931,1230.95009327,1230.88481426,1230.89010715
,1230.87952137,1231.65531673,1230.96597195,1230.89540005,1230.90026684
,1231.81353855,1230.92779179,1230.95362186,1230.8895422,1230.79194088
,1230.57260303,1231.86223507,1230.7466341,1231.61650219,1230.82306385
,1230.67493229,1231.85411739,1231.71177425,1231.93477955,1231.70351772
,1230.76491146,1231.81064434,1231.60824566,1230.98008633,1231.77472401
,1231.69349689,1231.79406185,1229.83505726,1230.8402113,1231.94473095
,1230.95023212,1231.76173954,1230.75425625,1230.75778484,1231.81537228
,1230.74380932,1231.88467541,1230.79659939,1230.81247807
,1230.77542782,1230.95079708,1229.85975742,1230.63957691,1230.91198254
,1230.67316799,1231.72123013,1230.87952137,1229.91092205,1231.98587475
,1230.77500172,1231.84296665,1230.9253931,1230.70139675,1231.71353855
,1230.82609692,1230.96081791,1230.82129955,1230.83195477,1231.71007938
,1230.64430485,1231.95531673,1230.89963245,1229.97090816,1231.7740202
,1231.89942417,1230.64853725,1230.60605526,1230.95552502,1229.89680767
,1230.61261692,1231.78354549,1230.9661108,1231.75122318,1230.69666882]
Tv=np.array(Tv)

In [ ]:
bndv=[[0.00430458,0.99999966]
,[0.00327041,0.99999976]
,[0.00443376,0.99999968]
,[0.00371765,0.99999956]
,[0.00369279,0.99999939]
,[0.00325139,0.99999966]
,[0.00355917,0.9999997,]
,[0.00368433,0.99999962]
,[0.00407579,0.99999973]
,[0.00371874,0.99999967]
,[0.0045768,0.99999955]
,[0.00453411,0.9999995]
,[0.00359326,0.99999974]
,[0.00408158,0.99999956]
,[0.00447334,0.9999996,]
,[0.00390814,0.99999954]
,[0.00431339,0.99999948]
,[0.00345201,0.99999955]
,[0.00347231,0.99999962]
,[0.0045965,0.99999972]
,[0.00373775,0.99999987]
,[0.00409947,0.99999958]
,[0.00415755,0.99999953]
,[0.00384899,0.99999955]
,[0.00413856,0.99999953]
,[0.00379368,0.99999963]
,[0.00372898,0.99999963]
,[0.00364474,0.99999962]
,[0.00390244,0.99999944]
,[0.00394446,0.99999977]
,[0.00360353,0.99999963]
,[0.00411487,0.99999954]
,[0.0037276,0.99999962]
,[0.00353985,0.99999938]
,[0.00355215,0.9999997]
,[0.0038372,0.99999971]
,[0.00451011,0.99999969]
,[0.00375653,0.99999978]
,[0.00344868,0.99999963]
,[0.00397121,0.99999962]
,[0.00379631,0.9999995,]
,[0.00393822,0.99999929]
,[0.00360932,0.99999956]
,[0.00375825,0.99999951]
,[0.00368746,0.99999952]
,[0.00386312,0.99999977]
,[0.00336534,0.99999962]
,[0.00359539,0.99999963]
,[0.00365147,0.99999977]
,[0.00426816,0.99999968]
,[0.00368713,0.99999962]
,[0.00379923,0.99999964]
,[0.00368826,0.9999996]
,[0.00377781,0.99999945]
,[0.00391344,0.99999973]
,[0.0042004,0.99999952]
,[0.00322609,0.99999961]
,[0.00387541,0.99999961]
,[0.00369848,0.99999957]
,[0.00375252,0.99999973]
,[0.00410567,0.99999959]
,[0.00369555,0.99999964]
,[0.00332288,0.99999957]
,[0.00357686,0.99999953]
,[0.00355824,0.9999996,]
,[0.0040256,0.99999964]
,[0.00348017,0.99999964]
,[0.00339818,0.99999949]
,[0.00358586,0.99999962]
,[0.00335975,0.99999975]
,[0.00319131,0.99999967]
,[0.00342015,0.99999969]
,[0.00460163,0.99999963]
,[0.00405083,0.99999971]
,[0.00367585,0.99999975]
,[0.00413251,0.99999968]
,[0.00321671,0.99999958]
,[0.00377343,0.99999982]
,[0.00376793,0.99999972]
,[0.00385406,0.99999976]
,[0.00432205,0.99999964]
,[0.00349574,0.99999955]
,[0.00373507,0.99999966]
,[0.00372223,0.99999963]
,[0.00375644,0.99999964]
,[0.00404044,0.99999943]
,[0.00457695,0.99999962]
,[0.00387039,0.99999969]
,[0.00406958,0.99999949]
,[0.00410755,0.99999948]
,[0.00377239,0.9999996]
,[0.00366549,0.99999964]
,[0.00454599,0.99999974]
,[0.00346909,0.99999963]
,[0.00439556,0.9999996,]
,[0.00337675,0.99999954]
,[0.00363312,0.99999958]
,[0.00408142,0.99999981]
,[0.0033241,0.99999963]
,[0.00406638,0.99999962]
,[0.00412449,0.99999944]
,[0.00371638,0.99999964]
,[0.0039663,0.99999959]
,[0.00356092,0.99999946]
,[0.00395115,0.99999955]
,[0.00347266,0.99999981]
,[0.00407838,0.99999977]
,[0.00448474,0.99999966]
,[0.00299448,0.99999973]
,[0.00332939,0.99999952]
,[0.00442228,0.99999984]
,[0.00390999,0.9999997]
,[0.0041279,0.99999976]
,[0.00454476,0.99999966]
,[0.00426373,0.99999961]
,[0.00334899,0.99999981]
,[0.00411946,0.99999961]
,[0.00344707,0.99999991]
,[0.00399475,0.99999965]]
bndv=np.array(bndv)

In [ ]:
print('mean invariant composition:', np.mean(bndv[:, 0]))
print('mean invariant temperature:', np.mean(Tv))

In [ ]:
from scipy.stats import gaussian_kde
def plot_contour(points, c='k', bw=.3):

    """
    Plot as set of KDE probability density contours for a set
    of points, typically corresponding to invariant locations

    Parameters
    ----------
    points : numpy array
        an array of compositions and temperatures representing
        invariant points or some other phase diagram feature
    c : color, optional
        color of the density contours
    bw : float, optional
        KDE bandwidth

    Returns
    -------

    Examples
    --------
    >>> import numpy as np
    >>> from pduq.invariant_calc import invariant_samples
    >>> from pduq.uq_plot import plot_contour
    >>> from pycalphad import Database
    >>> # load dbf file and raw parameter set.
    >>> dbf = Database('CU-MG_param_gen.tdb')
    >>> params = np.loadtxt('trace.csv', delimiter=',')
    >>> # find the set of invariant points
    >>> Tv, phv, bndv = invariant_samples(
    >>>     dbf, params, X=.2, P=101325, Tl=600, Tu=1400,
    >>>     comp='MG')
    >>> # define the 'points' array
    >>> points = np.zeros((len(Tv, 2)))
    >>> points[:, 0] = bndv[:, 1]
    >>> points[:, 1] = Tv
    >>> # plot the contour
    >>> plt.figure()
    >>> uq.plot_contour(points)
    """
    kernel = gaussian_kde(points.T)

    buf_mult =0.5

    xmin = points[:, 0].min()
    xmax = points[:, 0].max()
    buf = buf_mult*(xmax-xmin)
    xvec = np.linspace(xmin-buf, xmax+buf, 100)

    ymin = points[:, 1].min()
    ymax = points[:, 1].max()
    buf = buf_mult*(ymax-ymin)
    yvec = np.linspace(ymin-buf, ymax+buf, 100)

    X, Y = np.meshgrid(xvec, yvec)
    XY = np.vstack([X.ravel(), Y.ravel()])

    kernel = gaussian_kde(points.T, bw_method=bw)
    pdf = kernel(XY).T
    Z = pdf.reshape(X.shape)

    order = pdf.argsort()
    pdf_s = pdf[order]
    F = np.cumsum(pdf_s)
    F /= F[-1]

    Plvls = np.array([1-.9545, 1-.6827])
    boundaries = F.searchsorted(Plvls)
    lvls = pdf_s[boundaries]

    CS = plt.contour(X, Y, Z, lvls, colors=c, alpha=.7, linestyles=[':', '-'])
    labels = ["95% CI", "68% CI"]

    for ii in range(len(labels)):
        CS.collections[ii].set_label(labels[ii])

In [ ]:
points = np.zeros((len(Tv), 2))
points[:, 0] = bndv[:, 0]
points[:, 1] = Tv
c = sns.color_palette("Blues", 2)

plt.figure(figsize=(5, 3))

# plot the raw invariant points
plt.plot(points[:, 0], points[:, 1], 'k.',
         label="invariant\nsamples")

# plot KDE estimated uncertainty intervals
#labels = ["96% CI", "68% CI"]
plot_contour(points, c, 1)

plt.xlabel(r'$\mathrm{x_{Co}}$', fontsize="large")
plt.ylabel('T (K)', fontsize="large")
plt.tight_layout()
plt.legend()
plt.savefig('newkde.png',dpi=600)

In [ ]:
eqm, PhTm = mini(1240)
   # equ, PhTu = mini(1500)
print(np.squeeze(eqm.X.where(eqm.Phase == "FCC_A1").sel(component=comp).sum(dim='vertex').values))

In [ ]:
T=[1231.27027378,1231.52172565,1230.32912998,1231.21777096,1231.24247112
,1231.73880367,1231.51360798,1231.63061657,1231.30668964,1231.51600666
,1230.26857891,1230.21445065,1231.27302914,1230.50668964,1230.3502254
,1231.52835674,1230.66498089,1231.69286251,1231.71946583,1230.15270023
,1231.21953526,1231.78375378,1230.71494617,1230.96604137,1230.94317493
,1231.68947277,1230.80189228,1231.5981554,1230.70252666,1230.66194782
,1231.87888699,1231.1678751,1230.80365658,1231.6623045,1231.82066517
,1230.68368435,1229.9479723,1230.77189922,1231.44945889,1230.92842617
,1230.72426319,1230.67076931,1230.95009327,1230.88481426,1230.89010715
,1230.87952137,1231.65531673,1230.96597195,1230.89540005,1230.90026684
,1231.81353855,1230.92779179,1230.95362186,1230.8895422,1230.79194088
,1230.57260303,1231.86223507,1230.7466341,1231.61650219,1230.82306385
,1230.67493229,1231.85411739,1231.71177425,1231.93477955,1231.70351772
,1230.76491146,1231.81064434,1231.60824566,1230.98008633,1231.77472401
,1231.69349689,1231.79406185,1229.83505726,1230.8402113,1231.94473095
,1230.95023212,1231.76173954,1230.75425625,1230.75778484,1231.81537228
,1230.74380932,1231.88467541,1230.79659939,1230.81247807,1230
,1230.77542782,1230.95079708,1229.85975742,1230.63957691,1230.91198254
,1230.67316799,1231.72123013,1230.87952137,1229.91092205,1231.98587475
,1230.77500172,1231.84296665,1230.9253931,1230.70139675,1231.71353855
,1230.82609692,1230.96081791,1230.82129955,1230.83195477,1231.71007938
,1230.64430485,1231.95531673,1230.89963245,1229.97090816,1231.7740202
,1231.89942417,1230.64853725,1230.60605526,1230.95552502,1229.89680767
,1230.61261692,1231.78354549,1230.9661108,1231.75122318,1230.69666882]
T=np.array(T)
nn=T.shape[0]
print(nn)

In [ ]:
import math
comp = 'CO'
bbb=[]
ttt=[]
for ii in range(nn):
    eqm, PhTm = mini(T[ii])
    afb = np.squeeze(eqm.X.where(eqm.Phase == "FCC_A1").sel(component=comp).values)[1]
    if math.isnan(afb) == True:
        continue
    bbb.append(np.squeeze(eqm.X.where(eqm.Phase == "FCC_A1").sel(component=comp).values)[1])
    ttt.append(T[ii])

In [ ]:
ttt2=[1744.85,1745,1744.68,1743.59,1744.27,1745.15,1745.32,1744.11,1744.43,
      1744.74,1744.97,1744.37,1744.29,1743.6,1744.82,1744.28,1745.04,1744.72,
      1743.77,1744.63,1745.23,1744.81,1744.55,1744.51,1744.36,1744.11,1744.4,
      1744.11,1743.96,1745.26,1744.12,1744.54,1745.21,1745.68,1744.46,1743.54,
      1744.14,1744.76,1744.45,1744.36,1744.37,1745.52,1744.61,1744.55,1744.8,
      1743.55,1743.73,1744.24,1744.22,1744.34,1743.89,1744.48,1744.25,1743.11,
      1744.35,1744.13,1744.45,1743.89,1743.6,1743.97,1744.06,1744.97,1744.27,
      1745.29,1744.61,1743.34,1744.33,1744.14,1744.99,1743.77,1743.74,1743.99,
      1744.09,1743.55,1743.59,1743.72,1745.01,1744.17,1744.43,1744.23,1744.49,
      1744.14,1744.46,1743.88,1744.05,1744.4,1743.38,1743.81,1744.2,1743.84,
      1743.60873,1744.00672882, 1744.77073212, 1743.67073517, 1743.82768402, 
      1744.83667145, 1744.44157867, 1744.68898773, 1743.31351471, 
      1744.55329437, 1744.17427521, 1743.89280548, 1744.35792694, 
      1744.28354034, 1743.68490448, 1743.93231659, 1743.60752106, 
      1743.76310577, 1744.96122101, 1744.39607391, 1744.53040619, 
      1744.3366745, 1744.8936203, 1743.74757538, 1744.30479279, 
      1744.43394623, 1743.84512177, 1745.47781219, 1744.15111542, 1743.58844757]
ttt2=np.array(ttt2)

In [ ]:
mybnv = np.zeros((len(ttt2), 2))

In [ ]:
domainban = [0.994781,0.994782,0.994783,0.994783,0.994784,0.994785,0.994785,0.994785,0.994786,
             0.994786,0.994787,0.994787,0.994788,0.994788,0.994788,0.994788,0.994789,0.994789,
             0.994789,0.994789,0.994789,0.994789,0.994791,0.994791,0.994791,0.994791,0.994791,
             0.994791,0.994791,0.994791,0.994792,0.994792,0.994792,0.994793,0.994793,0.994793,
             0.994793,0.994793,0.994793,0.994793,0.994793,0.994794,0.994794,0.994794,0.994794,
             0.994794,0.994794,0.994795,0.994795,0.994795,0.994795,0.994795,0.994795,0.994795,
             0.994795,0.994795,0.994795,0.994796,0.994796,0.994796,0.994796,0.994796,0.994796,
             0.994796,0.994797,0.994797,0.994797,0.994798,0.994798,0.994798,0.994799,0.994799,
             0.994799,0.994799,0.994801,0.994801,0.994801,0.994801,0.994801,0.994802,0.994802,
             0.994802,0.994802,0.994803,0.994804,0.994805,0.994805,0.994806,0.994807,0.994808,
             0.994803,0.9947994333364296,0.9947905539789353,0.9948031327877772,0.9948057007651387,
             0.9947875689537214,0.9947982683122485,0.9947884268424535,0.9948041600855441,
             0.9947944635765154,0.9947940864242881,0.994800940743206,0.994796177662876,
             0.9947975143755035,0.9948046736810149,0.9948074122554788,0.9948078571781419,
             0.9948032697691382,0.9947840677416823,0.994795492075875,0.9947948749932904,
             0.9948001528187783,0.9947901423226733,0.994807138443585,0.9947935378051197,
             0.9947858185537888,0.9948017970876923,0.9947724195113905,0.9947980969695948,
             0.9948081994082464]
domainban=np.array(domainban)
nn=domainban.shape[0]
print(nn)

In [ ]:
for ii in range(nn):
    mybnv[ii, 0] = domainban[ii]

In [ ]:
print('mean invariant composition:', np.mean(mybnv[:, 0]))
print('mean invariant temperature:', np.mean(ttt2))

In [ ]:
from scipy.stats import gaussian_kde
def plot_contour(points, c='k', bw=0.3):

    """
    Plot as set of KDE probability density contours for a set
    of points, typically corresponding to invariant locations

    Parameters
    ----------
    points : numpy array
        an array of compositions and temperatures representing
        invariant points or some other phase diagram feature
    c : color, optional
        color of the density contours
    bw : float, optional
        KDE bandwidth

    Returns
    -------

    Examples
    --------
    >>> import numpy as np
    >>> from pduq.invariant_calc import invariant_samples
    >>> from pduq.uq_plot import plot_contour
    >>> from pycalphad import Database
    >>> # load dbf file and raw parameter set.
    >>> dbf = Database('CU-MG_param_gen.tdb')
    >>> params = np.loadtxt('trace.csv', delimiter=',')
    >>> # find the set of invariant points
    >>> Tv, phv, bndv = invariant_samples(
    >>>     dbf, params, X=.2, P=101325, Tl=600, Tu=1400,
    >>>     comp='MG')
    >>> # define the 'points' array
    >>> points = np.zeros((len(Tv, 2)))
    >>> points[:, 0] = bndv[:, 1]
    >>> points[:, 1] = Tv
    >>> # plot the contour
    >>> plt.figure()
    >>> uq.plot_contour(points)
    """
    kernel = gaussian_kde(points.T)

    buf_mult = 1

    xmin = points[:, 0].min()
    xmax = points[:, 0].max()
    buf = buf_mult*(xmax-xmin)
    xvec = np.linspace(xmin-buf, xmax+buf, 1000)

    ymin = points[:, 1].min()
    ymax = points[:, 1].max()
    buf = buf_mult*(ymax-ymin)
    yvec = np.linspace(ymin-buf, ymax+buf, 100)

    X, Y = np.meshgrid(xvec, yvec)
    XY = np.vstack([X.ravel(), Y.ravel()])

    kernel = gaussian_kde(points.T, bw_method=bw)
    pdf = kernel(XY).T
    Z = pdf.reshape(X.shape)

    order = pdf.argsort()
    pdf_s = pdf[order]
    F = np.cumsum(pdf_s)
    F /= F[-1]

    Plvls = np.array([1-.9545, 1-.6827])
    boundaries = F.searchsorted(Plvls)
    lvls = pdf_s[boundaries]

    CS = plt.contour(X, Y, Z, lvls, colors=c, alpha=.7, linestyles=[':', '-'])
    labels = ["95% CI", "68% CI"]

    for ii in range(len(labels)):
        CS.collections[ii].set_label(labels[ii])

In [ ]:
points = np.zeros((len(ttt2), 2))
points[:, 0] = mybnv[:, 0]
points[:, 1] = ttt2
c = sns.color_palette("Blues", 2)

plt.figure(figsize=(5, 3))

# plot the raw invariant points
plt.plot(points[:, 0], points[:, 1], 'k.',
         label="invariant\nsamples")

# plot KDE estimated uncertainty intervals
#labels = ["96% CI", "68% CI"]
plot_contour(points, c, 0.8)

plt.xlabel(r'$\mathrm{x_{Co}}$', fontsize="large")
plt.ylabel('T (K)', fontsize="large")

plt.tight_layout()
plt.legend()
plt.savefig('Figure224.jpg', bbox_inches = "tight",dpi=600)

相稳定性概率分布

In [ ]:
import numpy as np
from dask.distributed import Client
from distributed.deploy.local import LocalCluster
from pycalphad import Database, variables as v
from pduq.dbf_calc import eq_calc_samples
from pduq.uq_plot import get_phase_prob, plot_dist
import warnings
warnings.filterwarnings("ignore")

In [ ]:
c = LocalCluster(n_workers=8, threads_per_worker=1)
client = Client(c)
print(client)

In [ ]:
dbf = Database('ag-co_mcmc.tdb')
params = np.load('trace.npy')[:, -500:, :]
print(np.load('trace.npy').shape)
print(np.load('trace.npy')[:, -1, :].shape)
print(np.load('trace.npy')[:, -500:, :].shape)

In [ ]:
params = params.reshape(90000,18)
print(params.shape)

In [ ]:
# Define the equilibrium conditions including pressure
# (Pa), temperature (K), and molar composition Mg
conds = {v.P: 101325, v.T: 1740, v.X('CO'): 0.04}

# perform the equilibrium calculation for all parameter
# sets
eq = eq_calc_samples(dbf, conds, params, client=client)

In [ ]:
phaseregLL = [['LIQUID'],['FCC_A1', 'LIQUID'], 
              ['FCC_A1'], ['FCC_A1', 'HCP_A3'],['HCP_A3']]

# calculate the probability of non-zero phase fraction
for phaseregL in phaseregLL:
    prob = get_phase_prob(eq, phaseregL)
    print(phaseregL, ' ', np.squeeze(prob.values))

In [ ]:
import matplotlib.pyplot as plt
#coordD = {'X_CU':0.1}
coordD = {'T':1740, 'X_CO':0.04, 'component':'CO'}
#v.X('MG'): (0, 1, 0.01)
phaseregL = ['FCC_A1','LIQUID']
phase = 'FCC_A1'

# plot the phase fraction
plot_dist(eq, coordD, phaseregL, phase, typ='NP', figsize=(5, 3))
plt.savefig("NP3.png",dpi=600)
# plot the phase composition
plot_dist(eq, coordD, phaseregL, phase, typ='X', figsize=(5, 3))

plt.savefig("X3.png",dpi=600)
# plot molar Gibbs energy of the X-T-P point
plot_dist(eq, coordD, phaseregL, phase, typ='GM', figsize=(5, 3))
plt.savefig("GM3.png",dpi=600)
# chemical potential of the selected component
plot_dist(eq, coordD, phaseregL, phase, typ='MU', figsize=(5, 3))
plt.savefig("MU3.png",dpi=600)

In [ ]:
# Define the equilibrium conditions including pressure
# (Pa), temperature (K), and molar composition Mg
conds = {v.P: 101325, v.T: 1232, v.X('CO'): 0.004}

# perform the equilibrium calculation for all parameter
# sets
eq = eq_calc_samples(dbf, conds, params, client=client)

In [ ]:
phaseregLL = [['LIQUID'],['FCC_A1', 'LIQUID'], 
              ['FCC_A1'], ['FCC_A1', 'HCP_A3'],['HCP_A3']]

# calculate the probability of non-zero phase fraction
for phaseregL in phaseregLL:
    prob = get_phase_prob(eq, phaseregL)
    print(phaseregL, ' ', np.squeeze(prob.values))

In [ ]:
import matplotlib.pyplot as plt
#coordD = {'X_CU':0.1}
coordD = {'T':1232, 'X_CO':0.004, 'component':'CO'}
#v.X('MG'): (0, 1, 0.01)
phaseregL = ['FCC_A1','LIQUID']
phase = 'FCC_A1'

# plot the phase fraction
plot_dist(eq, coordD, phaseregL, phase, typ='NP', figsize=(5, 3))
plt.savefig("2NP3.png",dpi=600)
# plot the phase composition
plot_dist(eq, coordD, phaseregL, phase, typ='X', figsize=(5, 3))

plt.savefig("2X3.png",dpi=600)
# plot molar Gibbs energy of the X-T-P point
plot_dist(eq, coordD, phaseregL, phase, typ='GM', figsize=(5, 3))
plt.savefig("2GM3.png",dpi=600)
# chemical potential of the selected component
plot_dist(eq, coordD, phaseregL, phase, typ='MU', figsize=(5, 3))
plt.savefig("2MU3.png",dpi=600)